In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data=pd.read_csv("/kaggle/input/steam-store-games/steam.csv")
data=data.drop(["release_date", "steamspy_tags", "appid", "name", "english", "steamspy_tags"], axis=1)

In [2]:
platforms = data['platforms'].str.get_dummies(sep=';').astype('bool')
genres = data['genres'].str.get_dummies(sep=';').astype('bool')
categories = data['categories'].str.get_dummies(sep=';').astype('bool')

from sklearn.model_selection import train_test_split
y = data.owners
X = data.drop(['owners', 'categories', 'genres', 'platforms'], axis = 1)

X = pd.concat([X,platforms, genres, categories],axis=1)
X.info()
X.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 70 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   developer                   27075 non-null  object 
 1   publisher                   27075 non-null  object 
 2   required_age                27075 non-null  int64  
 3   achievements                27075 non-null  int64  
 4   positive_ratings            27075 non-null  int64  
 5   negative_ratings            27075 non-null  int64  
 6   average_playtime            27075 non-null  int64  
 7   median_playtime             27075 non-null  int64  
 8   price                       27075 non-null  float64
 9   linux                       27075 non-null  bool   
 10  mac                         27075 non-null  bool   
 11  windows                     27075 non-null  bool   
 12  Accounting                  27075 non-null  bool   
 13  Action                      270

,developer,publisher,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price,linux,...,Stats,Steam Achievements,Steam Cloud,Steam Leaderboards,Steam Trading Cards,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,VR Support,Valve Anti-Cheat enabled
0,Valve,Valve,0,0,124534,3339,17612,317,7.19,True,...,False,False,False,False,False,False,False,False,False,True
1,Valve,Valve,0,0,3318,633,277,62,3.99,True,...,False,False,False,False,False,False,False,False,False,True
2,Valve,Valve,0,0,3416,398,187,34,3.99,True,...,False,False,False,False,False,False,False,False,False,True
3,Valve,Valve,0,0,1273,267,258,184,3.99,True,...,False,False,False,False,False,False,False,False,False,True
4,Gearbox Software,Valve,0,0,5250,288,624,415,3.99,True,...,False,False,False,False,False,False,False,False,False,True


In [3]:
def point_estimation(r):
    rng = r.split('-')
    rng[0] = int(rng[0])
    rng[1] = int(rng[1])
    return (rng[0] + rng[1]) / 2

lst = []
for i in range(len(y)):
    lst.append(point_estimation(y[i]))

y = pd.DataFrame(lst)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, 
                                                                random_state=0)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, train_size=0.99, test_size=0.01, 
                                                                random_state=0)


In [4]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score


model1 = RandomForestRegressor(n_estimators=100, random_state=1, criterion='mse', n_jobs=-1, verbose=10)
model2 = LogisticRegression(solver='liblinear')
model3 = XGBRegressor(max_depth = 100, sampling_method = 'gradient_based', 
                      num_parallel_tree = 4, n_jobs=-1)
model4 = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=-1)
model5 = LinearRegression()

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, object_cols)
    ])

my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model1)
                             ])


scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')
print("MAE scores:\n", scores)

print("Average MAE score (across experiments):")
print(scores.mean())



my_pipeline.fit(X_train, y_train[0])
preds = my_pipeline.predict(X_valid)
score = mean_absolute_error(y_valid[0], preds)
print('MAE:', score)

/opt/conda/lib/python3.7/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   47.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Don

Using ThreadingBackend as joblib.Parallel backend instead of LokyBackend as the latter does not provide shared memory semantics.


[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.1s finished
/opt/conda/lib/python3.7/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   32.9s
[Parallel(n_jobs=-1)]: Don

Using ThreadingBackend as joblib.Parallel backend instead of LokyBackend as the latter does not provide shared memory semantics.


[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.3s finished
/opt/conda/lib/python3.7/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   34.3s
[Parallel(n_jobs=-1)]: Don

...

[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  6.7min


building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  7.3min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:    1.3s


MAE: 119534.09705582628


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.5s finished


In [6]:
X_train = pd.concat([X_train, X_valid], axis = 0)
y_train = pd.concat([y_train, y_valid], axis = 0)

my_pipeline.fit(X_train, y_train[0])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   51.4s


building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.3min


...

building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100


[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 10.2min


building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 11.1min finished


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['developer',
                                                   'publisher'])])),
                ('model',
                 RandomForestRegressor(n_jobs=-1, random_state=1, verbose=10))])

In [7]:
predicted = my_pipeline.predict(X_test)

ans = pd.DataFrame({
    'Predicted': predicted,
    'Y': y_test[0]
})

ans.head(50)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


,Predicted,Y
15928,10000.000000,10000.0
4113,10000.000000,10000.0
16779,10000.000000,10000.0
7450,10000.000000,10000.0
11939,10000.000000,35000.0
26427,10000.000000,10000.0
16202,10000.000000,10000.0
11505,70200.000000,10000.0
1188,100430.419681,75000.0
6955,170248.849206,35000.0
